**TODO :**
- Bigger img input / bigger mask
- mixup
- use oof to remove noisy annot

In [1]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

### Initialization

### Imports

In [2]:
import os
import sys
import torch
import warnings
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

sys.path.append("../code/")
# os.environ['CUDA_VISIBLE_DEVICES'] = "1,0"
warnings.simplefilter("ignore", UserWarning)

In [3]:
from training.main import k_fold

from utils.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
    update_overall_logs,
)

from params import *

### Load

In [4]:
df_info = pd.read_csv(DATA_PATH + f"HuBMAP-20-dataset_information.csv")
df_mask = pd.read_csv(DATA_PATH + "train.csv")

## Training

In [5]:
BATCH_SIZES = {
    "resnet18": 64,
    "resnet34": 32,
    "resnext50_32x4d": 32,
    "se_resnext50_32x4d": 32,
    "efficientnet-b0": 32,
    "efficientnet-b1": 32,
    "efficientnet-b2": 32,
    "efficientnet-b3": 16,
    "efficientnet-b4": 16,
    "efficientnet-b5": 16,
    "efficientnet-b6": 8,
}

In [6]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1

    device = "cuda" if torch.cuda.is_available() else "cpu"
    save_weights = True
    sampling_mode = 'centered' # chose between 'convhull', 'centered', 'random', 'visible'
    
    # Images
    tile_size = 512
    reduce_factor = 2
    on_spot_sampling = 0.99
    overlap_factor = 1.5

    img_dir = DATA_PATH + f"train_{tile_size}_red_{reduce_factor}"
    mask_dir = DATA_PATH + f"masks_{tile_size}_red_{reduce_factor}"

    # k-fold
    cv_column = "5fold"
    random_state = 0
    selected_folds = [0, 1, 2, 3, 4]

    # Model
    encoder = "efficientnet-b1"  # "resnet18" "resnext50_32x4d", "resnet34", "efficientnet-b5"
    decoder = "Unet"  # "Unet", "DeepLabV3Plus"
    use_bot = False
    use_fpn = False
    double_model = False
    use_mixstyle = False
    encoder_weights = "imagenet"
    num_classes = 1

    # Training
    loss = "BCEWithLogitsLoss"  # "SoftDiceLoss" / "BCEWithLogitsLoss"  / "lovasz"
    activation = "none" if loss == "lovasz" else "sigmoid"

    optimizer = "Adam"
    
    batch_size = BATCH_SIZES[encoder]
    
    if tile_size == 512:
        batch_size = batch_size // 2
        
    if batch_size >= 32:
        epochs = 50 
    elif batch_size >= 16:
        epochs = 40
    elif batch_size >= 6:
        epochs = 30
    else:
        epochs = 25
    
    iter_per_epoch = 5000
    lr = 1e-3
    swa_first_epoch = 50

    warmup_prop = 0.05
    val_bs = batch_size * 2

    first_epoch_eval = 0
    
    mix_proba = 0
    mix_alpha = 0.4
    
    if mix_proba > 0:
        epochs *= 3
    
    use_fp16 = True
    # external features : https://www.kaggle.com/baesiann/glomeruli-hubmap-external-1024x1024
    # download this and put folders in input/external_data
    # api command for download (requires kaggle api + credentials):
    # kaggle datasets download -d baesiann/glomeruli-hubmap-external-1024x1024
    use_external = 0.2 # percentage of sampling on external data (training only), None to disable
    
    oof_folder = None  # "../logs/2021-04-14/0/"
    loss_oof_weight = 0  # 0.5
    
    use_pl = 0.25
    pl_path = "../output/submission_0937.csv"
    
    if use_pl > 0:
        epochs += 10

In [7]:
DEBUG = False
log_folder = None

In [ ]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f"Logging results to {log_folder}")
    config_df = save_config(Config, log_folder + "config.json")
    create_logger(directory=log_folder, name="logs.txt")

metrics = k_fold(Config, log_folder=log_folder)

Logging results to ../logs/2021-04-29/0/
Creating in-memory dataset ...
Done in 68 seconds.

-------------   Fold 1 / 5  -------------

    -> 8757105 trainable parameters
    -> Validation images : ['2f6ecfcdf', 'b2dc8411c', '4ef6695ce'] 

Epoch 01/50 	 lr=4.0e-04	 t=246s	loss=0.223	val_loss=0.030 	 dice=0.8801
Epoch 02/50 	 lr=8.0e-04	 t=243s	loss=0.054	val_loss=0.010 	 dice=0.9115
Epoch 03/50 	 lr=9.9e-04	 t=256s	loss=0.041	val_loss=0.009 	 dice=0.9041
Epoch 04/50 	 lr=9.7e-04	 t=254s	loss=0.037	val_loss=0.007 	 dice=0.9246
Epoch 05/50 	 lr=9.5e-04	 t=246s	loss=0.035	val_loss=0.007 	 dice=0.9503
Epoch 06/50 	 lr=9.3e-04	 t=245s	loss=0.033	val_loss=0.007 	 dice=0.9519
Epoch 07/50 	 lr=9.1e-04	 t=242s	loss=0.032	val_loss=0.006 	 dice=0.9459
Epoch 08/50 	 lr=8.8e-04	 t=247s	loss=0.030	val_loss=0.006 	 dice=0.9527
Epoch 09/50 	 lr=8.6e-04	 t=253s	loss=0.029	val_loss=0.006 	 dice=0.9528
Epoch 10/50 	 lr=8.4e-04	 t=248s	loss=0.029	val_loss=0.006 	 dice=0.9483
Epoch 11/50 	 lr=8.2e-04	 t=2